In [1]:
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
    horizontal-align: middle;
}
h1,h2 {
    text-align: center;
    background-color: pink;
    padding: 20px;
    margin: 0;
    color: black;
    font-family: ariel;
   
}

h3 {
    text-align: center;
    border-style: solid;
    border-width: 3px;
    padding: 12px;
    margin: 0;
    color: black;
    font-family: ariel;
    border-radius: 80px;
    border-color: gold;
}

body, p {
    font-family: ariel;
    font-size: 15px;
    color: charcoal;
}
div {
    font-size: 14px;
    margin: 0;

}

h4 {
    padding: 0px;
    margin: 0;
    font-family: ariel;
    color: purple;
    
}

h1, h2, h3, h4, h5, h6 {
  background: #D4D4D4;
  border-top: 1px solid gray;
  border-bottom: 1px solid gray;
  font-weight: normal;
  padding-left: 2px;
  padding-right: 2px;
}
blockquote {
  background: #E6F2E6;
  border-left: 3px solid #408040;
  font-style: italic;
  padding-left: 5px;
  margin-left: 2px;
}
pre {
  background: #E0F0FF;
  border-left: 5px solid #55AAFF;
  padding: 5px;
  margin-left: 2px;
}
code {
  background: #E0F0FF;
  font-family: JetBrains Mono NL, Courier New, Monospaced;
  font-size: 0.9em;
}
table {
  border-spacing: 0px;
  border-right: 1px solid gray;
  border-bottom: 1px solid gray;
}
th, td {
  border-left: 1px solid gray;
  border-top: 1px solid gray;
}
th {
  background: #D4D4D4;
  font-weight: 600;
}
ul {
  margin-left-ltr: 20px;
  margin-right-rtl: 20px;
}
ol {
  margin-left-ltr: 25px;
  margin-right-rtl: 25px;
}
</style>
""")

In [1]:
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

import keras
import tensorflow as tf
from tensorflow.keras import models, layers, utils

from sklearn.metrics import accuracy_score, classification_report, recall_score, confusion_matrix, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
from tensorflow.keras.utils import to_categorical

from sklearn.feature_selection import SelectKBest, chi2
import re
import unicodedata
import tkinter as tk
from tkinter import StringVar,OptionMenu, Label, Text, Entry


import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report


class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        c = ['\b|', '\b/', '\b-', '\b\\'] 
        print(c[epoch % 4], end='')
    def on_epoch_end(self, epoch, logs=None):
        print('\b', end='')

# DISEASE PREDICTION USING SYMPTOMS

In [2]:
descriptions = pd.read_csv('data/symptom_Description.csv')
precautions = pd.read_csv('data/symptom_precaution.csv')

In [3]:
df = pd.read_csv('data/dataset.csv')
print(df.head(2))


            Disease   Symptom_1              Symptom_2              Symptom_3  \
0  Fungal infection     itching              skin_rash   nodal_skin_eruptions   
1  Fungal infection   skin_rash   nodal_skin_eruptions    dischromic _patches   

              Symptom_4 Symptom_5 Symptom_6 Symptom_7 Symptom_8 Symptom_9  \
0   dischromic _patches       NaN       NaN       NaN       NaN       NaN   
1                   NaN       NaN       NaN       NaN       NaN       NaN   

  Symptom_10 Symptom_11 Symptom_12 Symptom_13 Symptom_14 Symptom_15  \
0        NaN        NaN        NaN        NaN        NaN        NaN   
1        NaN        NaN        NaN        NaN        NaN        NaN   

  Symptom_16 Symptom_17  
0        NaN        NaN  
1        NaN        NaN  


In [4]:
severity = pd.read_csv('data/Symptom-severity.csv')
print(severity.head())

                Symptom  weight
0               itching       1
1             skin_rash       3
2  nodal_skin_eruptions       4
3   continuous_sneezing       4
4             shivering       5


## Cleaning the data

In [5]:
df.isna().sum()
cols = df.columns
data = df[cols].values.flatten()
cols = df.columns

s = pd.Series(data)
s = s.str.strip()
s = s.values.reshape(df.shape)

df = pd.DataFrame(s, columns=df.columns)

df = df.fillna(0)
df.head()
df = df.fillna(0)
df.head(2)

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
vals = df.values
symptoms = severity['Symptom'].unique()

In [7]:
df.describe()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
count,4920,4920,4920,4920,4920,4920,4920,4920,4920,4920,4920,4920,4920,4920,4920,4920,4920,4920
unique,41,34,48,54,51,39,33,27,22,23,22,19,12,9,5,4,4,2
top,Fungal infection,vomiting,vomiting,fatigue,high_fever,0,0,0,0,0,0,0,0,0,0,0,0,0
freq,120,822,870,726,378,1206,1986,2652,2976,3228,3408,3726,4176,4416,4614,4680,4728,4848


In [8]:
ordered_symptoms = []

for disease_row in df['Disease'].index:
    temp_list = []
    for symptom_col in df.columns[1:]:
        if df.loc[disease_row, symptom_col] == 0:
            break
        temp_list.append(df.loc[disease_row, symptom_col])
    ordered_symptoms.append(temp_list)



In [9]:
diseases = sorted(df['Disease'].unique())
descs = descriptions.sort_values(by='Disease')
pre_c = precautions.sort_values(by='Disease')

In [10]:
ordered_cautions = []

for i in range(len(pre_c['Disease'])):
    temp_list = []
    for k in range(1,5):
        temp_list.append(pre_c.iloc[i][k])
    ordered_cautions.append(temp_list)
    
#Empty lists to append sorted values to in the symptoms data
ordered_symptoms = []

for i in range(len(df['Disease'])):
    temp_list = []
    for k in range(1,17):
        if df.iloc[i][k] == 0:
            break
        temp_list.append(df.iloc[i][k])
    ordered_symptoms.append(temp_list)

C:\Users\patty\AppData\Local\Temp\ipykernel_10956\1815331482.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  temp_list.append(pre_c.iloc[i][k])
C:\Users\patty\AppData\Local\Temp\ipykernel_10956\1815331482.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df.iloc[i][k] == 0:
C:\Users\patty\AppData\Local\Temp\ipykernel_10956\1815331482.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  temp_list.append(df.iloc[i][k])


In [11]:
disease_dict = {}

for i in range(len(df['Disease'])):
    symptoms_list = []
    for k in range(len(df.columns)):
        if df.iloc[i][k] == 0 or df.iloc[i][k] in disease_dict.keys():
            continue
        symptoms_list.append(df.iloc[i][k])
    disease_dict[df['Disease'][i]] = symptoms_list
    
sorted_keys = sorted(disease_dict.keys())
symptoms_list = []

for i in range(len(sorted_keys)):
    #Appending the list containing the symptoms from our disease dictionary
    symptoms_list.append(disease_dict[sorted_keys[i]])

C:\Users\patty\AppData\Local\Temp\ipykernel_10956\352069636.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df.iloc[i][k] == 0 or df.iloc[i][k] in disease_dict.keys():
C:\Users\patty\AppData\Local\Temp\ipykernel_10956\352069636.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  symptoms_list.append(df.iloc[i][k])


In [12]:
#Assembling everything into a single dataframe
df = pd.DataFrame({"Diseases":diseases,"Descriptions":descs['Description'],
"Precautions":ordered_cautions, "Symptoms":symptoms_list})

In [13]:
df.head()


,Diseases,Descriptions,Precautions,Symptoms
9,(vertigo) Paroymsal Positional Vertigo,Benign paroxysmal positional vertigo (BPPV) is...,"[lie down, avoid sudden change in body, avoid ...","[vomiting, headache, nausea, spinning_movement..."
23,AIDS,Acquired immunodeficiency syndrome (AIDS) is a...,"[avoid open cuts, wear ppe if possible, consul...","[muscle_wasting, patches_in_throat, high_fever..."
11,Acne,"Acne vulgaris is the formation of comedones, p...","[bath twice, avoid fatty spicy food, drink ple...","[skin_rash, pus_filled_pimples, blackheads, sc..."
31,Alcoholic hepatitis,"Alcoholic hepatitis is a diseased, inflammator...","[stop alcohol consumption, consult doctor, med...","[vomiting, yellowish_skin, abdominal_pain, swe..."
2,Allergy,An allergy is an immune system response to a f...,"[apply calamine, cover area with bandage, nan,...","[continuous_sneezing, shivering, chills, water..."


In [14]:
medicines = pd.read_csv('testingData/disease_medicines.csv')
riskfactors = pd.read_csv('testingData/disease_riskFactors.csv')

In [15]:
data = pd.merge(medicines, riskfactors, left_on = 'Disease_ID', right_on= 'DID', how = 'inner')

data = data[['Medicine_Name', 'DNAME', 'RISKFAC', 'PRECAU']]
def list_agg(series):
    return list(series)
# Group by 'DNAME' (disease name) and aggregate 'Medicine_Name' and 'RISKFAC' into lists
new_data = data.groupby('DNAME').agg({
    'Medicine_Name': list_agg,
    'RISKFAC': list_agg,
    'PRECAU':list_agg
}).reset_index()
new_data.columns = ['Disease', 'Medicine_Name', 'Risk_Factors', 'Precautions']

new_data = pd.DataFrame( new_data, columns=new_data.columns)
new_data.head()

,Disease,Medicine_Name,Risk_Factors,Precautions
0,ADHD,[no medicine available (consult doctor)],"[Exposure to environmental toxins, family hist...","[protein-rich breakfast, walk to school if pos..."
1,Acne,[Isoniazid (antibiotics)],"[Being teenager, Hormonal changes, Friction or...","[Bath twice, avoid fatty spicy food, drink ple..."
2,Adenomyosis,[no medicine available (consult doctor)],"[Prior uterine surgery, such as a C-section or...",[No precaution]
3,Alzheimer's,[cholinesterase inhibitors],"[Lack of exercise, Obesity, Smoking, High BP, ...","[Stopping smoking, keeping alcohol to a minimu..."
4,Amnesia,[no medicine available (consult doctor)],"[Brain surgery, head injury or trauma, Stroke,...","[Avoid heavy use of alcohol, drugs, Stay menta..."


In [16]:
new_data['Disease']= new_data['Disease'].str.lower()
new_data['Disease']= new_data['Disease'].str.replace(r'[^\w\s]', '')

df['Diseases']= df['Diseases'].str.lower()
df['Diseases']= df['Diseases'].str.replace(r'[^\w\s\']', '')

In [20]:
data = pd.merge(new_data, df, left_on= 'Disease', right_on= 'Diseases', how = 'left')
data.drop(columns = ['Diseases', "Precautions_y"], inplace = True )
nan_description = "No Description found for this disease"
nan_symptom = "No Symptoms found for this disease"

data['Descriptions'].fillna(nan_description, inplace=True)
data['Symptoms'].fillna(nan_symptom, inplace = True)
data.head()
import pandas as pd
import os

directory = 'Disease_data'
if not os.path.exists(directory):
    os.makedirs(directory)

# Specify the file path
file_path = os.path.join(directory, 'disease_data.csv')

# Assuming your DataFrame is named df
# Write DataFrame to CSV
data.to_csv(file_path, index=False)

print(f"DataFrame has been exported to {file_path}")



DataFrame has been exported to Disease_data\disease_data.csv


In [18]:
df_model= pd.read_csv('data/dataset.csv')
symptoms_data2 = df_model.copy()

In [19]:
df_model.isna().sum()
cols = df_model.columns
data = df_model[cols].values.flatten()
cols = df_model.columns

s = pd.Series(data)
s = s.str.strip()
s = s.values.reshape(df_model.shape)

df_model = pd.DataFrame(s, columns=df_model.columns)

df_model = df_model.fillna(0)
df_model.head()
df_model = df_model.fillna(0)
df_model.head(2)

vals = df_model.values
symptoms = severity['Symptom'].unique()
#encode the symptoms with their severity weight
len(symptoms)# 132 unique symptoms
for i in range(len(symptoms)):
    vals[vals == symptoms[i]] = severity[severity['Symptom'] == symptoms[i]]['weight'].values[0]
d = pd.DataFrame(vals, columns=cols)

d = d.replace('dischromic _patches', 0)
d = d.replace('spotting_ urination',0)
symptoms_data= d.replace('foul_smell_of urine',0)
symptoms_data.head()


,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,1,3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Fungal infection,3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Fungal infection,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Fungal infection,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Fungal infection,1,3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### EXPLORATORY DATA ANALYSIS

We countplot to visualise the frequency distribution of the symptoms to understand the distribution of symptoms across all diseases. It helps identify common symptoms that may occur across multiple diseases and less common symptoms that may be more specific to certain diseases.Symptoms with a weight of 0 are ignored as they do not provide meaningful information about disease manifestation. Symptoms with higher weights are indicative of their greater importance in diagnosing specific diseases.

From the distribution below, it's notable that symptom 4 emerges as the most prevalent across all diseases, indicating its widespread occurrence and potential relevance in various medical conditions. Conversely, symptoms with weights 1 and 7 appear to be least common, possibly suggesting their rarity or specificity to certain diseases.

In [ ]:
labels = symptoms_data['Disease']
features = symptoms_data.drop(columns= ['Disease'])

class_counts = labels.value_counts()

group1 = class_counts[:len(class_counts)//2]
group2 = class_counts[len(class_counts)//2:]

plt.figure(figsize=(10, 6))
sns.barplot(x=group1.index, y=group1.values)
plt.title('Class Distribution (Group 1)')
plt.xlabel('Class')
plt.ylabel('Number of Samples')
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(10, 6))
sns.barplot(x=group2.index, y=group2.values)
plt.title('Class Distribution (Group 2)')
plt.xlabel('Class')
plt.ylabel('Number of Samples')
plt.xticks(rotation=45)
plt.show()



In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(data=symptoms_data.melt(id_vars=['Disease'], value_vars=symptoms_data.columns[1:]), x='value')
plt.title('Frequency Distribution of Symptoms')
plt.xlabel('Symptom')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
plt.show()



Correlation analysis helps identify if certain symptoms tend to occur together. Understanding symptom correlations can assist in providing insights into potential underlying relationships between symptoms and diseases, thus aiding in explaining som decisions made by the model.

In [ ]:
# Calculate correlation matrix
corr_matrix = symptoms_data.iloc[:, 1:].corr()

# Heatmap to visualize correlation between symptoms
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Between Symptoms')
plt.show()


Understanding the distribution of diseases in the dataset is crucial for assessing class imbalance. It helps ensure that the dataset is adequately representative of all diseases and can guide model evaluation and selection of appropriate evaluation metrics.

In [ ]:
# Countplot to visualize frequency distribution of diseases
plt.figure(figsize=(10, 6))
sns.countplot(data=symptoms_data, x='Disease')
plt.title('Frequency Distribution of Diseases')
plt.xlabel('Disease')
plt.ylabel('Frequency')
plt.xticks(rotation=90)
plt.show()


##  Why is this a multiclass multilabel classification problem ?
 
1. **Multiclass Classification**: The model's task is to classify instances (patients) into one of multiple disease classes. In this case, there are multiple diseases that could be predicted based on the given symptoms. For example, given a set of symptoms, the model might predict "Fungal Infection" and "GERD" as the two most likely diseases.

2. **Multilabel Classification**: Each instance (patient) can belong to multiple classes (diseases) simultaneously. In this scenario, it's possible for a patient to exhibit symptoms that are indicative of more than one disease. For instance, a patient might have symptoms that correspond to both "Fungal Infection" and "Food Allergy".

Therefore, with the task of predicting the two most likely diseases from a set of given symptoms, the problem is a multiclass multilabel classification problem. The model needs to learn the relationships between symptoms and diseases to make accurate predictions about which diseases are most likely given the observed symptoms.


### FIRST DATASET

In [ ]:
df_model2 = pd.read_csv('testingData/Training.csv')
df_model2.drop(columns=['Unnamed: 133'], inplace=True)
label_encoder = LabelEncoder()


labels = df_model2['prognosis']
col_labels = labels.unique()
features = df_model2.iloc[:, :-1]

print("Shape of labels: ", labels.shape)
print("Shape of features:", features.shape)

labels_encoded = label_encoder.fit_transform(labels)
labels = to_categorical(labels_encoded)
features = np.reshape(features, (features.shape[0], -1))
print("Shape of reshaped features: ", features.shape)
print("Shape of reshaped labels: ", labels.shape)

X_train, X_test, y_train, y_test =train_test_split(features, labels, test_size=0.2, random_state=42)


### SECOND DATASET

In [ ]:
symptoms_data = symptoms_data.copy()
#move the first column symptoms_data['Disease'] to be the last column
disease_column = symptoms_data.pop('Disease')
label_encoder = LabelEncoder()
# Reinsert the 'Disease' column at the end
symptoms_data['Disease'] = disease_column


features = symptoms_data.iloc[:, :-1]
labels = symptoms_data['Disease']
col_labels = labels.unique()

print("Shape of labels: ", labels.shape)
print("Shape of features:", features.shape)

labels_encoded = label_encoder.fit_transform(labels)
labels = to_categorical(labels_encoded)
print("Shape of reshaped labels: ", labels.shape)
features = np.reshape(features, (features.shape[0], -1))
print("Shape of reshaped features: ", features.shape)

X_train, X_test, y_train, y_test =train_test_split(features, labels, test_size=0.2, random_state=42)

### MODEL CREATION

In [ ]:


def create_model(optimizer='adam', dropout_rate=0.3, num_layers=2, units_per_layer=64):
    model = models.Sequential()
    model.add(layers.Dense(units_per_layer, activation='relu', input_shape=(features.shape[1],)))
    for _ in range(num_layers - 1):  # Add additional hidden layers if num_layers > 1
        model.add(layers.Dense(units_per_layer, activation='tanh', kernel_regularizer=tf.keras.regularizers.l1(0.002)))
        model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(labels.shape[1], activation='softmax'))
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy' ,keras.metrics.Recall(),keras.metrics.Precision()])
    return model


In [ ]:
network = create_model()
network.fit(X_train,y_train,epochs = 30, batch_size= 32,verbose=0,callbacks=[CustomCallback()])
network.summary()

In [ ]:

loss, accuracy, precision, recall_metric = network.evaluate(X_test, y_test)

# Print the evaluation results
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
print("Test Precision:", precision)
print("Test Recall (Keras metric):", recall_metric)


y_pred = network.predict(X_test)
y_pred_array = np.argmax(y_pred_raw,axis=1)
y_pred_labels = np.reshape(y_pred_array, (y_pred_array.shape[0], -1))
y_pred_one_hot = to_categorical(y_pred_labels)




In [ ]:
# Convert one-hot encoded y_test into 1D array
y_test_indices = y_test.argmax(axis=1)
y_pred_binary = y_pred_one_hot.argmax(axis=1)
conf_matrix = confusion_matrix(y_test_indices, y_pred_binary)

plt.figure(figsize=(10,7))
sns.heatmap(conf_matrix, annot=True, fmt='d',cmap='Blues', xticklabels=col_labels, yticklabels=col_labels)
plt.xlabel("Predicted")
plt.ylabel("Actual")

In [ ]:
model = create_model()
initial_weights = model.get_weights()
K = 4
num_val_samples = len(X_train) // K
num_epochs = 20
all_recall_histories = []

for i in range(K):
    print('processing fold', i)
    a, b = i * num_val_samples, (i + 1) * num_val_samples
    val_data = X_train[a:b]
    val_targets = y_train[a:b]
    

    partial_train_data = np.concatenate([X_train[:a], X_train[b:]], axis=0)
    partial_train_targets = np.concatenate([y_train[:a], y_train[b:]], axis=0)

    # Reset model weights before training each fold
    model.set_weights(initial_weights)
    
    # Train the model
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=32, verbose=0,callbacks=[CustomCallback()])

    recall_history = history.history['accuracy']
    all_recall_histories.append(recall_history)


In [ ]:
average_recall_history = [np.mean([x[i] for x in all_recall_histories])for i in range(num_epochs)]
plt.plot(range(1, len(average_recall_history) + 1), average_recall_history)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
hist = history.history
hist.keys()


In [ ]:
# Model accuracy
plt.plot(hist['accuracy'])
plt.plot(hist['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['accuracy', 'val_accuracy'])
plt.show()

In [ ]:
# Model Losss
plt.plot(hist['loss'])
plt.plot(hist['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss', 'val_loss'])
plt.show()

In [ ]:
# generate classification report using predictions for categorical model
from sklearn.metrics import classification_report, accuracy_score

categorical_pred = np.argmax(model.predict(X_test), axis=1)

print('Results for Categorical Model')
print(accuracy_score(y_test, y_pred_one_hot))
print(classification_report(y_test, y_pred_one_hot))

# BASELINE MODEL :MACHINE  LEARNING
### Multinomial Naive Bayes classifier.

In [ ]:

clf_df= pd.read_csv('trainingData/symptom-disease-train-dataset.csv')
# Split data into features and labels
features = clf_df['text']
labels = clf_df['label']
clf_df.head()

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=420)

# Vectorize the text data using TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Initialize and train the MNB classifier
clf = MultinomialNB()
clf.fit(X_train_tfidf, y_train)
# Predictions MLB
y_pred_nb = clf.predict(X_test_tfidf)

# Initialize and train the SVM classifier
svm = SVC(kernel='linear')
svm.fit(X_train_tfidf, y_train)
# Predictions MLB
y_pred_svm= svm.predict(X_test_tfidf)


# Classification report
#print(classification_report(y_test, y_pred))


In [ ]:
# Initialize and train the MNB classifier
clf = MultinomialNB()
clf.fit(X_train_tfidf, y_train)
# Predictions MLB
y_pred_nb = clf.predict(X_test_tfidf)

# Initialize and train the SVM classifier
svm = SVC(kernel='linear')
svm.fit(X_train_tfidf, y_train)
# Predictions MLB
y_pred_svm= svm.predict(X_test_tfidf)


# Classification report
#print(classification_report(y_test, y_pred))



# Evaluate the classifier
from sklearn.metrics import precision_score, recall_score, f1_score
# Calculate metrics for Naive Bayes
nb_accuracy = accuracy_score(y_test, y_pred_nb)
nb_precision = precision_score(y_test, y_pred_nb, average='weighted')
nb_recall = recall_score(y_test, y_pred_nb, average='weighted')
nb_f1 = f1_score(y_test, y_pred_nb, average='weighted')

# Calculate metrics for SVM
svm_accuracy = accuracy_score(y_test, y_pred_svm)
svm_precision = precision_score(y_test, y_pred_svm, average='weighted')
svm_recall = recall_score(y_test, y_pred_svm, average='weighted')
svm_f1 = f1_score(y_test, y_pred_svm, average='weighted')

# Create a DataFrame to display results
results_df = pd.DataFrame({
    'Algorithm': ['Multinomial Naive Bayes', 'Support Vector Machine (SVM)'],
    'Accuracy': [nb_accuracy, svm_accuracy],
    'Precision': [nb_precision, svm_precision],
    'Recall': [nb_recall, svm_recall],
    'F1-score': [nb_f1, svm_f1]
})

results_df

In [4]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import WordNetLemmatizer, PorterStemmer

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\patty\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\patty\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\patty\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
# Load the dataset
df = pd.read_csv('trainingData/symptom-disease-train-dataset.csv')

# Initialize WordNet lemmatizer
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

# Function to preprocess text
def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    
    # Lemmatize tokens
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]
    
     # Stem tokens
    #stemmed_tokens = [stemmer.stem(word) for word in lemmatized_tokens]
    
    # Join tokens back into text
    preprocessed_text = ' '.join(lemmatized_tokens)
    
    return preprocessed_text

# Apply preprocessing to the 'text' column
df['cleaned_text'] = df['text'].apply(preprocess_text)


# Print the cleaned text
print(df['cleaned_text'].iloc[0])
df.drop(columns=['text'], inplace=True)
df.head()

migraine headache . ca n't sleep . whole body shaking shivering . feel dizzy sometimes .


,label,cleaned_text
0,308,migraine headache . ca n't sleep . whole body ...
1,35,asthma get wheezing breathing problem . also f...
2,798,Signs symptom primary ovarian insufficiency si...
3,149,"cough , high_fever , breathlessness , family_h..."
4,596,"chill , vomiting , high_fever , sweating , hea..."


In [6]:

# Features and labels
features =df['cleaned_text']
labels = df['label']



# Tokenization
max_words = 10000  # Maximum number of words to tokenize
tokenizer = keras.preprocessing.text.Tokenizer(num_words=max_words)
features= features.tolist()
tokenizer.fit_on_texts(features)
sequences = tokenizer.texts_to_sequences(features)

# Padding
vocab_size = len(tokenizer.word_index) + 1
max_len = 100  # Maximum sequence length


features = keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_len)
print("Length of features: ",len(features[1]))

# Encoding labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

encoder = OneHotEncoder(sparse=False)
labels_encoded = labels_encoded.reshape((labels.shape[0],1))
labels_onehot= encoder.fit_transform(labels_encoded)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels_onehot, test_size=0.2, random_state=42)
# Number of unique classes for output layer
num_classes = len(label_encoder.classes_)
#print("X_train",X_train[1])
#print("x_train for example word: ",feature)
print("first label of the dataset",labels_onehot.shape[1])

Length of features:  100
first label of the dataset 866


In [ ]:


example_text = "I have been having migraines and headaches. I can't sleep. My whole body is shaking and shivering. I feel dizzy sometimes."
tokenizer.fit_on_texts(example_text)
sequence = tokenizer.texts_to_sequences(example_text)
feature = keras.preprocessing.sequence.pad_sequences(sequence, maxlen=max_len)

print("x_train for example word: ",feature)



#DECODE THE PREDICTED OUTPUT

labels_encoded = labels_onehot.reshape(-1, labels_onehot.shape[1])

# Inverse transform one-hot encoded labels
labels_encoded_inverse = encoder.inverse_transform(labels_encoded)

# Inverse transform label encoding
labels_decoded = label_encoder.inverse_transform(labels_encoded_inverse)

# Print the decoded label for the first sample
print(labels_decoded[0])


In [ ]:
from keras.models import Sequential
from keras import layers

embedding_dim = 50

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=max_len))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
               metrics=['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])
model.summary()

In [ ]:
embedding_dim = 50

model3 = Sequential()
model3.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=max_len))
model3.add(layers.GlobalMaxPool1D())
model3.add(layers.Dense(64, activation='relu'))
model3.add(layers.Dense(num_classes, activation='softmax'))
model3.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])
model3.summary()

history = model3.fit(X_train, y_train,
                    epochs=30,
                    verbose=False,
                    validation_data=(X_test, y_test),
                    batch_size=64)
loss3, accuracy3, precision3, recall3 = model.evaluate(X_test, y_test)

print("Test Loss:", loss3)
print("Test Accuracy:", accuracy3)
print("Test Precision:", precision3)
print("Test Recall (Keras metric):", recall3)


In [7]:


example_text = "having migraines and headaches. I can't sleep.  shaking and shivering. dizzy ."
tokenizer.fit_on_texts(example_text)
sequence = tokenizer.texts_to_sequences(example_text)
feature = keras.preprocessing.sequence.pad_sequences(sequence, maxlen=max_len)


example_prediction = model.predict(feature)
print(example_prediction)

#decode the prediction
labels_encoded = example_prediction.reshape(-1, labels_onehot.shape[1])

# Inverse transform one-hot encoded labels
labels_encoded_inverse = encoder.inverse_transform(labels_encoded)

# Inverse transform label encoding
labels_decoded = label_encoder.inverse_transform(labels_encoded_inverse)

# Print the decoded label for the first sample
print(labels_decoded[0])

index_237 = df[df['label'] == 394].index
df['cleaned_text'].iloc[index_237]

NameError: name 'model' is not defined

In [ ]:
# Model architecture
def create_model(optimizer='adam', dropout_rate=0.3, num_layers=2, units_per_layer=128):
    model = models.Sequential()
    model.add(layers.Dense(units_per_layer, activation='relu', input_shape=(max_len,)))
    for _ in range(num_layers - 1):
        model.add(layers.Dense(units_per_layer, activation='tanh', kernel_regularizer=tf.keras.regularizers.l1(0.002)))
        model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(num_classes, activation='softmax'))
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])
    return model

In [8]:

network_FNN = create_model()
network_FNN.fit(X_train,y_train,epochs = 30, batch_size= 64,verbose=0,callbacks=[CustomCallback()])
network_FNN.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               12928     
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 866)               111714    
                                                                 
Total params: 141154 (551.38 KB)
Trainable params: 141154 (551.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
loss, accuracy, precision, recall_metric = network_FNN.evaluate(X_test, y_test)

# Print the evaluation results
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
print("Test Precision:", precision)
print("Test Recall (Keras metric):", recall_metric)
predictions =  network_FNN.predict(X_test)


36/36 [==============================] - 0s 2ms/step - loss: 3.2764 - accuracy: 0.6877 - recall: 0.6442 - precision: 0.9758
Test Loss: 3.2763755321502686
Test Accuracy: 0.6876663565635681
Test Precision: 0.6441881060600281
Test Recall (Keras metric): 0.975806474685669
36/36 [==============================] - 0s 2ms/step
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[4.6381343e-02 1.2698401e-05 7.7875275e-06 ... 7.2490649e-05
  1.5786033e-04 3.9775255e-06]
 [2.5944884e-05 1.3674504e-06 2.3945158e-06 ... 4.1473149e-06
  2.4086883e-06 1.4218813e-06]]


In [ ]:


example_text = "having migraines and headaches. I can't sleep.  shaking and shivering. dizzy ."
tokenizer.fit_on_texts(example_text)
sequence = tokenizer.texts_to_sequences(example_text)
feature = keras.preprocessing.sequence.pad_sequences(sequence, maxlen=max_len)

example_prediction = model.predict(feature)
print(example_prediction)

#decode the prediction
labels_encoded = example_prediction.reshape(-1, labels_onehot.shape[1])

# Inverse transform one-hot encoded labels
labels_encoded_inverse = encoder.inverse_transform(labels_encoded)

# Inverse transform label encoding
labels_decoded = label_encoder.inverse_transform(labels_encoded_inverse)

# Print the decoded label for the first sample
print(labels_decoded[0])

### WORD EMBEDDINGS: GLOBAL VECTORS FOR WORD REPRESENTATION


In [ ]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath, encoding = 'utf-8') as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]
    return embedding_matrix

In [ ]:
embedding_dim = 50
embedding_matrix = create_embedding_matrix('trainingData/glove.6B/glove.6B.50d.txt', tokenizer.word_index, embedding_dim )

In [ ]:
#94% of the vocabulary is covered by the pretrained model, which is a good coverage of our vocabulary.
nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
nonzero_elements / vocab_size

In [ ]:
embedding_dim = 50

model = Sequential()
model.add(layers.Embedding(vocab_size, 
                           embedding_dim, 
                           input_length=max_len,
                          trainable= True))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(128, activation='tanh'))
model.add(layers.Dense(num_classes, activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])
model.summary()

In [ ]:
history = model.fit(X_train, y_train,
                    epochs=30,
                    verbose=False,
                    validation_data=(X_test, y_test),
                    batch_size=64)
loss, accuracy, precision, recall_metric = model.evaluate(X_test, y_test)

# Print the evaluation results
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
print("Test Precision:", precision)
print("Test Recall (Keras metric):", recall_metric)


In [ ]:

example_text = " migraines and headaches. I can't sleep.  shaking and shivering. dizzy ."
tokenizer.fit_on_texts(example_text)
sequence = tokenizer.texts_to_sequences(example_text)
feature = keras.preprocessing.sequence.pad_sequences(sequence, maxlen=max_len)


example_prediction = model.predict(feature)
print(example_prediction)

#decode the prediction
labels_encoded = example_prediction.reshape(-1, labels_onehot.shape[1])

# Inverse transform one-hot encoded labels
labels_encoded_inverse = encoder.inverse_transform(labels_encoded)

# Inverse transform label encoding
labels_decoded = label_encoder.inverse_transform(labels_encoded_inverse)

# Print the decoded label for the first sample
print(labels_decoded[0])

## Convolutional Neural Networks (CNN)

In [ ]:
embedding_dim = 100

model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(layers.Conv1D(256,5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dropout(0.3))
model.add(layers.Dense(128, activation='tanh'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(num_classes, activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])
model.summary()

In [ ]:
history = model.fit(X_train, y_train,
                    epochs=30,
                    verbose=False,
                    validation_data=(X_test, y_test),
                    batch_size=64)
loss, accuracy, precision, recall_metric = model.evaluate(X_test, y_test)

# Print the evaluation results
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
print("Test Precision:", precision)
print("Test Recall (Keras metric):", recall_metric)


In [ ]:


example_text = "I have been having migraines and headaches. I can't sleep. My whole body is shaking and shivering. I feel dizzy sometimes."
tokenizer.fit_on_texts(example_text)
sequence = tokenizer.texts_to_sequences(example_text)
feature = keras.preprocessing.sequence.pad_sequences(sequence, maxlen=max_len)



example_prediction = model.predict(feature)
print(example_prediction)

#decode the prediction
labels_encoded = example_prediction.reshape(-1, labels_onehot.shape[1])

# Inverse transform one-hot encoded labels
labels_encoded_inverse = encoder.inverse_transform(labels_encoded)

# Inverse transform label encoding
labels_decoded = label_encoder.inverse_transform(labels_encoded_inverse)

# Print the decoded label for the first sample
print(labels_decoded[0])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
# Define the model
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(layers.Conv1D(256, 5, activation='relu'))  # Changed filter size to 5
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dropout(0.5))  # Added dropout for regularization
model.add(layers.Dense(128, activation='tanh'))
model.add(layers.Dropout(0.5))  # Added dropout for regularization
model.add(layers.Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),  # Added learning rate scheduling
              loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

# Print model summary
model.summary()

# Train the model
history = model.fit(X_train, y_train,
                    epochs=20,  # Increased epochs for more training
                    batch_size=64,  # Changed batch size
                    validation_data=(X_test, y_test),
                    callbacks=[EarlyStopping(patience=3)])  # Early stopping to prevent overfitting

# Evaluate the model
loss, accuracy, recall, precision = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
print("Test Recall:", recall)
print("Test Precision:", precision)

In [ ]:
example_text = "migraine headache . ca n't sleep . whole body shaking shivering . feel dizzy sometimes ."
tokenizer.fit_on_texts(example_text)
sequence = tokenizer.texts_to_sequences(example_text)
feature = keras.preprocessing.sequence.pad_sequences(sequence, maxlen=max_len)


example_prediction = model.predict(feature)
print(example_prediction)

#decode the prediction
labels_encoded = example_prediction.reshape(-1, labels_onehot.shape[1])

# Inverse transform one-hot encoded labels
labels_encoded_inverse = encoder.inverse_transform(labels_encoded)

# Inverse transform label encoding
labels_decoded = label_encoder.inverse_transform(labels_encoded_inverse)

# Print the decoded label for the first sample
print(labels_decoded[0])
filtered_df = df[df['label'] == 284]
df.head()

## LSTM-based recurrent neural network 

In [ ]:
# Build the LSTM model
embedding_dim = 50  # Set the dimension of word embeddings
model = Sequential([
    layers.Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len),
   layers.LSTM(units=128),
    layers.Dense(units=64, activation='relu'),
    layers.Dropout(0.1),
    layers.Dense(units=num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

# Define early stopping to prevent overfitting
#early_stopping = EarlyStopping(monitor='val_oss', patience=3)

num_classes# 866
X_train.shape#(4507, 100)
y_test.shape #(1127, 866)





In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=30, batch_size=64, validation_split=0.2)

# Evaluate the model
loss, accuracy, precision, recall_metric = model.evaluate(X_test, y_test)

# Print the evaluation results
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
print("Test Precision:", precision)
print("Test Recall (Keras metric):", recall_metric)

In [ ]:
example_text = "migraines and headaches. can't sleep.  body shaking and shivering. dizzy sometimes."
tokenizer.fit_on_texts(example_text)
sequence = tokenizer.texts_to_sequences(example_text)
feature = keras.preprocessing.sequence.pad_sequences(sequence, maxlen=max_len)

prediction = model.predict(feature)

# Reshape example_prediction to a 2D array


# Flatten labels_encoded using ravel()
labels_encoded = prediction.ravel()

# Inverse transform one-hot encoded labels
labels_encoded_inverse = encoder.inverse_transform(labels_encoded.reshape(-1, 866))
labels_encoded_inverse= labels_encoded_inverse.ravel()
# Inverse transform label encoding
labels_decoded = label_encoder.inverse_transform(labels_encoded_inverse)

# Print the decoded label for the first sample
print(labels_decoded[0])


In [ ]:
#user input
user_input = "Opps, i feel so bad i want to die, i missed my periods  and my whole body is literally nervous and bleeding. I have a terrible migraines and headaches. I can't sleep.  My whole body is shaking and shivering. I feel dizzy sometimes."
example_text =  preprocess_text(user_input)
print("Cleaned and tokenized user input: ",example_text)
tokenizer.fit_on_texts(example_text)
sequence = tokenizer.texts_to_sequences(example_text)
feature = keras.preprocessing.sequence.pad_sequences(sequence, maxlen=max_len)

example_prediction = model.predict(feature)

#decode the prediction
labels_encoded = example_prediction.reshape( labels_onehot.shape[1],-1)

# Inverse transform one-hot encoded labels
labels_encoded_inverse = encoder.inverse_transform(labels_encoded)

# Inverse transform label encoding
labels_decoded = label_encoder.inverse_transform(labels_encoded_inverse)

# Print the decoded label for the first sample
print(labels_decoded[0])

In [9]:





import numpy as np
#input_size = 150, hidden_size= 128, output_size = num_classes=866
# initialize weights and biases
def init_params(input_size, hidden_size, output_size):
    np.random.seed(0)
    W1 = np.random.randn(hidden_size, input_size) * 0.01
    b1 = np.zeros((hidden_size, 1))
    W2 = np.random.randn(output_size, hidden_size) * 0.01
    b2 = np.zeros((output_size, 1))
    parameters = [W1, b1,  W2, b2]
    return parameters

parameters= init_params(max_len,128,num_classes)

def ReLU(Z,alpha=0.1):
    return np.maximum(alpha*Z, Z)

def softmax(Z):
    exp_Z = np.exp(Z - np.max(Z, axis=0))  # Subtract the maximum value
    return exp_Z / (np.sum(exp_Z, axis=0)+1e-15)

import numpy as np

def categorical_cross_entropy_loss(A2, Y):
    """
    Arguments:
    A2 -- probability vector corresponding to the predicted probabilities of a sample belonging to each class, shape (m, num_classes)
    Y -- true "label" vector, where each value is 0 or 1 for each class, shape (m, num_classes)
    Returns:
    loss -- the categorical cross-entropy loss
    """
    
    m = Y.shape[0]  # Number of samples
    epsilon = 1e-15  # Small value to avoid taking the logarithm of zero

    # Clip A to avoid log(0), and clip 1-A to avoid log(1)
    A_clipped = np.clip(A2, epsilon, 1 - epsilon)
    Y= Y.T
    # Compute the loss
    loss = -(1 / m) * np.sum(Y * np.log(A_clipped))

    return loss


def forward_propagation(X, parameters):
    W1, b1, W2, b2 = parameters
    Z1 = np.dot(W1, X.T) + b1
    A1 = ReLU(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = softmax(Z2)
    cache = {"Z1": Z1, "A1": A1, "Z2": Z2, "A2": A2}
    return A2, cache

def backward_propagation(parameters, cache, X, Y):
    m = Y.shape[0]
    W1, b1, W2, b2 = parameters
    Z1 = cache["Z1"]
    A1 = cache["A1"]
    Z2 = cache["Z2"]
    A2 = cache["A2"]
    dA2 = (A2 - Y.T)
    dZ2 = dA2 * (A2 * (1-A2))
    dW2 = (1/m) * np.dot(dZ2, A1.T)
    db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = dA1 * (A1 > 0)
    dW1 = (1/m) * np.dot(dZ1, X)
    db1 = (1/m) * np.sum(dZ1, axis=1, keepdims=True)
    gradients = {"dW1": dW1, "db1": db1, "dW2": dW2, "db2": db2}
    return gradients

def update_parameters(parameters, gradients, learning_rate):
    W1, b1, W2, b2 = parameters
    dW1 = gradients["dW1"]
    db1 = gradients["db1"]
    dW2 = gradients["dW2"]
    db2 = gradients["db2"]
    W1 = W1 - learning_rate*dW1
    b1 = b1 - learning_rate*db1
    W2 = W2 - learning_rate*dW2
    b2 = b2 - learning_rate*db2
    parameters = (W1, b1, W2, b2)
    return parameters

# train the neural network
def train(X, Y, hidden_layer_size, num_iterations, learning_rate):
    parameters = init_params(X.shape[1], hidden_layer_size, num_classes)
    for i in range(num_iterations):
        A2, cache = forward_propagation(X, parameters)
        loss = categorical_cross_entropy_loss(A2, Y)
        gradients = backward_propagation(parameters, cache, X, Y)
        parameters = update_parameters(parameters, gradients, learning_rate)
        if i % 1000 == 0:
            print(f"Iteration {i}: Loss = {loss}")
    return parameters

parameters = train(X_train, y_train, hidden_layer_size=128, num_iterations=3500, learning_rate=0.1)

MemoryError: Unable to allocate 29.8 MiB for an array with shape (866, 4507) and data type float64

In [12]:
# predict the labels for new data
def predict(X, parameters):
    A2, _ = forward_propagation(X, parameters)
    predictions= A2.T
    return predictions

predictions = predict(X_test, parameters)

def accuracy(y_true, y_pred):
   
    # Convert predicted probabilities to class labels
    y_pred_labels = np.argmax(y_pred, axis=1)
    
    # Convert true labels from one-hot encoding to class labels
    y_true_labels = np.argmax(y_true, axis=1)
    
    # Compute accuracy
    correct_predictions = np.sum(y_pred_labels == y_true_labels)
    total_samples = y_true.shape[0]
    acc = correct_predictions / total_samples
    print(f"predicted outcome: {y_pred_labels[:10]}True outcome: {y_true_labels[:10]}")
    return acc
accuracy_pred = accuracy(y_test,predictions)
accuracy_pred


predicted outcome: [ 23 329 374 611 358 165 102 826 223 320]True outcome: [ 23 329 374 611 358 165 848 415 223 320]


# USER INTERPHASE

In [168]:
import tkinter as tk
from tkinter import scrolledtext
from tkinter import messagebox
from tkinter import StringVar, OptionMenu, Text, Entry, Button, Label

# Function to handle prediction based on chosen model
def predict():
    selected_model = model_var.get()
    selected_symptoms = [symptom_var.get() for symptom_var in symptom_vars]
    typed_symptoms = userinput.get()
    
    # Perform prediction based on the selected model
    if selected_model == "SVM":
        # Call SVM prediction function
        prediction_result = svm_predict(selected_symptoms, typed_symptoms)
    elif selected_model == "FNN":
        # Call FNN prediction function
        prediction_result = fnn_predict(selected_symptoms, typed_symptoms)
    elif selected_model == "CNN":
        # Call CNN prediction function
        prediction_result = cnn_predict(selected_symptoms, typed_symptoms)
    else:
        prediction_result = "Please select a model"
    
    # Display prediction result
    result_text.delete('1.0', tk.END)  # Clear previous result
    result_text.insert(tk.END, prediction_result)

# Function to handle prediction using Support Vector Machine (SVM)
def svm_predict(symptoms , userinput):
    input_symptoms = " "
    if all(symptom == "None" for symptom in symptoms) and userinput == "None":
        return "OPPS!!", "ENTER  SYMPTOMS PLEASE"
    else:
        input_symptoms = " ".join(symptom for symptom in symptoms if symptom != "None")
        if userinput != "None":
            input_symptoms += ", " + userinput
        
        print(input_symptoms)
        text = preprocess_text(input_symptoms)
    
        # Tokenize and pad the sequence
        sequence = tokenizer.texts_to_sequences([text])
        feature = pad_sequences(sequence, maxlen=max_len)

        # Make prediction
        prediction = model.predict(feature)
      
        # Flatten labels_encoded using ravel()
        labels_encoded = prediction.ravel()

        # Inverse transform one-hot encoded labels
        labels_encoded_inverse = encoder.inverse_transform(labels_encoded.reshape(-1, 866))
        labels_encoded_inverse= labels_encoded_inverse.ravel()
        # Inverse transform label encoding
        labels_decoded = label_encoder.inverse_transform(labels_encoded_inverse)
      
        # Display the predicted disease
        predicted_disease = labels_decoded[0]
        return "Disease Prediction", f"The predicted disease is for symptoms, {input_symptoms} is : {predicted_disease}"
    
    


# Function to handle prediction using Feedforward Neural Network (FNN)
def fnn_predict(symptoms, userinput):
    # Code for FNN prediction
    return "Prediction using FNN model"

# Function to handle prediction using Convolutional Neural Network (CNN)
def cnn_predict(symptoms, userinput):
    # Code for CNN prediction
    return "Prediction using CNN model"

root = tk.Tk()
root.title("Disease Prediction From Symptoms")

# Styling
bg_color = "#f0f0f0"
font_style = ("Helvetica", 12)

# Main Frame
main_frame = tk.Frame(root, bg=bg_color)
main_frame.pack(padx=20, pady=20)

# Header
header_label = tk.Label(main_frame, text="Disease Prediction From Symptoms", font=("Helvetica", 18), bg=bg_color)
header_label.grid(row=0, column=0, columnspan=2, pady=10)

# Symptoms Section
symptoms_label = tk.Label(main_frame, text="Choose symptoms or type your symptoms:", font=font_style, bg=bg_color)
symptoms_label.grid(row=1, column=0, columnspan=2, pady=(0, 5))

symptoms_options = ["None", "rush", " bleeding", "Blisters", "tired", "Symptom 5", "Symptom 6", "Symptom 7"]
Symptom1 = StringVar()
Symptom1.set(None)
Symptom2 = StringVar()
Symptom2.set(None)
Symptom3 = StringVar()
Symptom3.set(None)
Symptom4 = StringVar()
Symptom4.set(None)
Symptom5 = StringVar()
Symptom5.set(None)
Symptom6 = StringVar()
Symptom6.set(None)
Symptom7 = StringVar()
Symptom7.set(None)
symptom_vars = [StringVar() for _ in range(7)]
for i in range(7):
    tk.Label(main_frame, text=f"Symptom {i+1}:", font=font_style, bg=bg_color).grid(row=i+2, column=0, pady=5, padx=(0, 10))
    OptionMenu(main_frame,symptom_vars[i], *symptoms_options).grid(row=i+2, column=1, pady=5)

# userInput
tk.Label(main_frame, text="Input Symptoms:", font=font_style, bg=bg_color).grid(row=9, column=0, pady=5, padx=(0, 10))
userinput = StringVar()
Entry(main_frame, textvariable=userinput).grid(row=9, column=1, pady=5)

# Model Selection
model_label = tk.Label(main_frame, text="Select prediction model:", font=font_style, bg=bg_color)
model_label.grid(row=10, column=0, columnspan=2, pady=(10, 5))

model_options = ["SVM", "FNN", "CNN"]
model_var = StringVar()
model_var.set(model_options[0])  # Set default model
OptionMenu(main_frame, model_var,*model_options).grid(row=11, column=0, columnspan=2)

# Predict Button
predict_button = tk.Button(main_frame, text="Predict", command=predict, bg="#4CAF50", fg="white", font=font_style)
predict_button.grid(row=12, column=0, columnspan=2, pady=(10, 0))

# Result Section
result_label = tk.Label(main_frame, text="Prediction Result:", font=font_style, bg=bg_color)
result_label.grid(row=13, column=0, columnspan=2, pady=(20, 5))

result_text = Text(main_frame, height=2, width=50)
result_text.grid(row=14, column=0, columnspan=2, pady=(0, 10))

root.mainloop()



   rush   , bdsjklx;zs'
1/1 [==============================] - 0s 60ms/step
